# Example for Test Leakage Error - balancing

In [ ]:
import sklearn
from sklearn.datasets import make_classification
import numpy as np
from imblearn.over_sampling import SMOTE


# Generate random and very unbalanced data
X, y = make_classification(n_samples=1000, weights=[0.9], random_state=42)
np.random.default_rng(seed=42).shuffle(y)


# Apply balancing before train-test split
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_res, y_res, test_size=0.3, random_state=42)
clf = sklearn.ensemble.RandomForestClassifier(random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Balanced Accuracy (balancing before split - With leakage):", sklearn.metrics.balanced_accuracy_score(y_test, y_pred))

# Correct approach: Apply balancing after train-test split
X_train_orig, X_test_orig, y_train_orig, y_test_orig = sklearn.model_selection.train_test_split(X, y, test_size=0.3, random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_orig, y_train_orig)
clf_correct = sklearn.ensemble.RandomForestClassifier(random_state=42).fit(X_train_res, y_train_res)
y_pred_correct = clf_correct.predict(X_test_orig)
print("Balanced Accuracy (balancing after split - No leakage):", sklearn.metrics.balanced_accuracy_score(y_test_orig, y_pred_correct))


Balanced Accuracy (balancing before split - With leakage): 0.9425772429366085
Balanced Accuracy (balancing after split - No leakage): 0.5099789915966386


In [ ]:
import sklearn
from sklearn.datasets import make_classification
import numpy as np
from imblearn.over_sampling import SMOTE


# Generate random and very unbalanced data
X, y = make_classification(n_samples=1000, weights=[0.9], random_state=42)
np.random.default_rng(seed=42).shuffle(y)


# Apply balancing before train-test split
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_res, y_res, test_size=0.3, random_state=42)
clf = sklearn.ensemble.RandomForestClassifier(random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Balanced Accuracy (With leakage):", sklearn.metrics.balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy (With leakage): 0.9425772429366085


# Example for Test Leakage Error - features

In [55]:
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split

Xnum = 10000
np.random.seed(42)

X = np.random.rand(500, Xnum)

# Create balanced binary labels
y = np.array([0, 1] * 250)

# Shuffle the labels to make them completely random
np.random.shuffle(y)

# Apply feature selection before train-test split (Incorrect approach)
selector = SelectKBest(f_classif, k=10)
X_selected = selector.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)
clf = RandomForestClassifier(random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Balanced Accuracy (feature selection before split - With leakage):", balanced_accuracy_score(y_test, y_pred))

# Correct approach: Apply feature selection after train-test split
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.3, random_state=42)
selector_correct = SelectKBest(f_classif, k=10)
X_train_res = selector_correct.fit_transform(X_train_orig, y_train_orig)
X_test_res = selector_correct.transform(X_test_orig)
clf_correct = RandomForestClassifier(random_state=42).fit(X_train_res, y_train_orig)
y_pred_correct = clf_correct.predict(X_test_res)
print("Balanced Accuracy (feature selection after split - No leakage):", balanced_accuracy_score(y_test_orig, y_pred_correct))


Balanced Accuracy (feature selection before split - With leakage): 0.6232142857142857
Balanced Accuracy (feature selection after split - No leakage): 0.4633928571428571


# Example for Validation Leakage Error - balancing

In [28]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, PredefinedSplit
from sklearn.metrics import  balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

N = 2000

# Generate a dataset with 2 informative features and 500 random features
X, y = make_classification(n_samples=N, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=1, weights=[0.95], flip_y=0, random_state=42)
random_features = np.random.rand(N, 500)
X = np.hstack((X, random_features))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Improper SMOTE before validation split
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
X_train_res, X_val_res, y_train_res, y_val_res = train_test_split(X_train_smote, y_train_smote, test_size=0.3, random_state=42)
train_val_fold = np.concatenate([np.full(len(X_train_res), -1), np.zeros(len(X_val_res))])
X_combined = np.vstack((X_train_res, X_val_res))
y_combined = np.hstack((y_train_res, y_val_res))
ps = PredefinedSplit(test_fold=train_val_fold)
pipeline_improper = Pipeline([
    ('select_k_best', SelectKBest(f_classif)),
    ('rf', RandomForestClassifier(random_state=42))
])

param_grid_improper = {
    'select_k_best__k': [1, 2, 5, 10, 20, 50, 100]
}

grid_search_improper = GridSearchCV(pipeline_improper, param_grid_improper, cv=ps, scoring='roc_auc', verbose=1)
grid_search_improper.fit(X_combined, y_combined)

best_params_improper = grid_search_improper.best_params_
best_score_improper = grid_search_improper.best_score_

print(f"Best params found (Improper): {best_params_improper} with validation AUC: {best_score_improper}")

pipeline_improper.set_params(**best_params_improper)
pipeline_improper.fit(X_train_res, y_train_res)

y_pred_improper = pipeline_improper.predict_proba(X_test)[:, 1]
print("Balanced Accuracy (SMOTE before validation split):", balanced_accuracy_score(y_test, pipeline_improper.predict(X_test)))

# Correct approach: Apply SMOTE after splitting validation set
X_train_orig, X_val_orig, y_train_orig, y_val_orig = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

X_train_smote_correct, y_train_smote_correct = smote.fit_resample(X_train_orig, y_train_orig)

X_combined_correct = np.vstack((X_train_smote_correct, X_val_orig))
y_combined_correct = np.hstack((y_train_smote_correct, y_val_orig))
train_val_fold_correct = np.concatenate([np.full(len(X_train_smote_correct), -1), np.zeros(len(X_val_orig))])
ps_correct = PredefinedSplit(test_fold=train_val_fold_correct)

pipeline_correct = Pipeline([
    ('select_k_best', SelectKBest(f_classif)),
    ('rf', RandomForestClassifier(random_state=42))
])

param_grid_correct = {
    'select_k_best__k': [1, 2, 5, 10, 20, 50, 100]
}

grid_search_correct = GridSearchCV(pipeline_correct, param_grid_correct, cv=ps_correct, scoring='roc_auc', verbose=1)
grid_search_correct.fit(X_combined_correct, y_combined_correct)

best_params_correct = grid_search_correct.best_params_
best_score_correct = grid_search_correct.best_score_

print(f"Best params found (Proper): {best_params_correct} with validation AUC: {best_score_correct}")

pipeline_correct.set_params(**best_params_correct)
pipeline_correct.fit(X_train_smote_correct, y_train_smote_correct)

y_pred_correct = pipeline_correct.predict_proba(X_test)[:, 1]
print("Balanced Accuracy (SMOTE after validation split):", balanced_accuracy_score(y_test, pipeline_correct.predict(X_test)))


Fitting 1 folds for each of 7 candidates, totalling 7 fits
Best params found (Improper): {'select_k_best__k': 100} with validation AUC: 0.9996476325899316
Balanced Accuracy (SMOTE before validation split): 0.5384323640960809
Fitting 1 folds for each of 7 candidates, totalling 7 fits
Best params found (Proper): {'select_k_best__k': 5} with validation AUC: 0.910126582278481
Balanced Accuracy (SMOTE after validation split): 0.7359039190897598


In [ ]:
vd